In [ ]:
import random
import pandas as pd
from scipy import stats

import numpy as np

from sklearn.metrics import f1_score
from collections import defaultdict

import time
import warnings
warnings.filterwarnings('ignore')

from matplotlib import pyplot as plt
from matplotlib.ticker import PercentFormatter

import torch
import torch.nn as nn

import dataloader as dataloader
import nn_model as nn_model
import imputation as imp
from baseline_utils import BaselineUtilTensor

from captum.attr import IntegratedGradients, DeepLift
from collections import defaultdict
import shap

rs = 42
np.random.seed(rs)
torch.manual_seed(rs)
random.seed(rs)

In [ ]:
batch_size = 64
num_epochs = 200
learning_rate = 4e-3
N = 100
shap_sample_size = 10
#possible blur/mean/zero
imputation_typ = 'blur'

In [ ]:
def integrated_gradients(X, baseline, model):
    start_time = time.time()
    ig = IntegratedGradients(model)
    X = X.cuda()
    baseline = baseline.cuda()
    attr = ig.attribute(X, baseline, target=0)
    print("--- '%.2f' seconds computation time ---" % (time.time() - start_time))
    return attr

In [ ]:
def integrated_gradients_mdb(X, model, X_min, X_max, columns):
    start_time = time.time()
    ig = IntegratedGradients(model)
    attr = []
    but = BaselineUtilTensor()
    for observation in X:
        cur_baseline = but.create_max_dist_baseline(observation, X_min, X_max, columns).unsqueeze(0).cuda()
        attr.append(ig.attribute(observation.unsqueeze(0).cuda(), cur_baseline, target=0))
        
    print("--- '%.2f' seconds computation time ---" % (time.time() - start_time))
    return torch.cat(attr)

In [ ]:
def integrated_gradients_avg(X, baseline, model, sample_size=10):
    start_time = time.time()
    attr = torch.zeros(X.shape)
    attr = attr.cuda()
    ig = IntegratedGradients(model)
    for i in range(sample_size):
        idx = np.random.randint(baseline.shape[0], size=1)
        sample = torch.from_numpy(baseline[idx,:])
        sample = sample.cuda()
        X = X.cuda()
        attr += ig.attribute(X, sample, target=0)
        
    print("--- '%.2f' seconds computation time ---" % (time.time() - start_time))
    return (attr/sample_size)

In [ ]:
def deep_lift(X, baseline, model):
    start_time = time.time()
    dl = DeepLift(model)
    X = X.cuda()
    baseline = baseline.cuda()
    attr = dl.attribute(X, baseline, target=0)
    print("--- '%.2f' seconds computation time ---" % (time.time() - start_time))
    return attr

In [ ]:
def deep_lift_mdb(X, model, X_min, X_max, columns):
    start_time = time.time()
    dl = DeepLift(model)
    attr = []
    but = BaselineUtilTensor()
    for observation in X:
        cur_baseline = but.create_max_dist_baseline(observation, X_min, X_max, columns).unsqueeze(0).cuda()
        attr.append(dl.attribute(observation.unsqueeze(0).cuda(), cur_baseline, target=0))
    print("--- '%.2f' seconds computation time ---" % (time.time() - start_time))
    return torch.cat(attr)

In [ ]:
def deep_lift_avg(X, baseline, model, sample_size=10):
    start_time = time.time()
    attr = torch.zeros(X.shape)
    attr = attr.cuda()
    dl = DeepLift(model)
    for i in range(sample_size):
        idx = np.random.randint(baseline.shape[0], size=1)
        sample = torch.from_numpy(baseline[idx,:])
        sample = sample.cuda()
        X = X.cuda()
        attr += dl.attribute(X, sample, target=0)
        
    print("--- '%.2f' seconds computation time ---" % (time.time() - start_time))
    return (attr/sample_size)

In [ ]:
def deep_shap(X, baseline, model):
    start_time = time.time()
    X = X.cuda()
    baseline = baseline.cuda()
    explainer = shap.DeepExplainer(model, baseline)
    shap_values = explainer.shap_values(X, ranked_outputs=1)
    print("--- '%.2f' seconds computation time ---" % (time.time() - start_time))
    return shap_values if len(shap_values) == X.shape[0] else shap_values[0][0]

In [ ]:
def deep_shap_mdb(X, model, X_min, X_max, columns):
    start_time = time.time()
    attr_list = []
    but = BaselineUtilTensor()
    for observation in X:
        cur_baseline = but.create_max_dist_baseline(observation, X_min, X_max, columns).unsqueeze(0).cuda()
        explainer = shap.DeepExplainer(model, cur_baseline)
        shap_values = explainer.shap_values(observation.unsqueeze(0).cuda(), ranked_outputs=1)
        if len(shap_values) == 1:
            attr_list.append(shap_values)
        else:
            attr_list.append(shap_values[0][0])
        
    print("--- '%.2f' seconds computation time ---" % (time.time() - start_time))
    return np.concatenate(attr_list, axis=0)

In [ ]:
def deep_shap_avg(X, baseline, model, sample_size=10):
    start_time = time.time()
    attr = np.zeros(X.shape)
    X = X.cuda()
    for i in range(sample_size):
        idx = np.random.randint(baseline.shape[0], size=1)
        sample = torch.from_numpy(baseline[idx,:])
        sample = sample.cuda()     
        explainer = shap.DeepExplainer(model, sample)
        shap_values = explainer.shap_values(X, ranked_outputs=1)
        if len(shap_values) == X.shape[0]:
            attr += shap_values
        else:
            attr += shap_values[0][0]
        
    print("--- '%.2f' seconds computation time ---" % (time.time() - start_time))
    return (attr/sample_size)

In [ ]:
def calc_ig_attr(X_test_c_tens, black_baseline, blurred_baseline, uniform_baseline, gaussian_baseline, 
                train_baseline, model, X_min, X_max, columns):
    print("Start integrated gradients for correct predictions with black_baseline")
    bb_attr = integrated_gradients(X_test_c_tens, black_baseline, model)
    print("Start integrated gradients for correct predictions with max_dist_baseline")
    mdb_attr = integrated_gradients_mdb(X_test_c_tens, model, X_min, X_max, columns)
    print("Start integrated gradients for correct predictions with blurred_baseline")
    blb_attr = integrated_gradients_avg(X_test_c_tens, blurred_baseline, model)
    print("Start integrated gradients for correct predictions with uniform_baseline")
    ub_attr = integrated_gradients_avg(X_test_c_tens, uniform_baseline, model)
    print("Start integrated gradients for correct predictions with gaussian_baseline")
    gb_attr = integrated_gradients_avg(X_test_c_tens, gaussian_baseline, model)
    print("Start integrated gradients for correct predictions with train_baseline")
    tb_attr = integrated_gradients_avg(X_test_c_tens, train_baseline, model)
    return bb_attr, mdb_attr, blb_attr, ub_attr, gb_attr, tb_attr

In [ ]:
def calc_dl_attr(X_test_c_tens, black_baseline, blurred_baseline, uniform_baseline, gaussian_baseline, 
                train_baseline, model, X_min, X_max, columns):
    print("Start DeepLIFT for correct predictions with black_baseline")
    bb_attr = deep_lift(X_test_c_tens, black_baseline, model)
    print("Start DeepLIFT for correct predictions with max_dist_baseline")
    mdb_attr = deep_lift_mdb(X_test_c_tens, model, X_min, X_max, columns)
    print("Start DeepLIFT for correct predictions with blurred_baseline")
    blb_attr = deep_lift_avg(X_test_c_tens, blurred_baseline, model)
    print("Start DeepLIFT for correct predictions with uniform_baseline")
    ub_attr = deep_lift_avg(X_test_c_tens, uniform_baseline, model)
    print("Start DeepLIFT for correct predictions with gaussian_baseline")
    gb_attr = deep_lift_avg(X_test_c_tens, gaussian_baseline, model)
    print("Start DeepLIFT for correct predictions with train_baseline")
    tb_attr = deep_lift_avg(X_test_c_tens, train_baseline, model)
    return bb_attr, mdb_attr, blb_attr, ub_attr, gb_attr, tb_attr

In [ ]:
def calc_ds_attr(X_test_c_tens, black_baseline, blurred_baseline, uniform_baseline, gaussian_baseline, 
                train_baseline, model, X_min, X_max, columns):
    print("Start DeepSHAP for correct predictions with black_baseline")
    bb_attr = deep_shap(X_test_c_tens, black_baseline, model)
    print("Start DeepSHAP for correct predictions with max_dist_baseline")
    mdb_attr = deep_shap_mdb(X_test_c_tens, model, X_min, X_max, columns)
    print("Start DeepSHAP for correct predictions with blurred_baseline")
    blb_attr = deep_shap_avg(X_test_c_tens, blurred_baseline, model)
    print("Start DeepSHAP for correct predictions with uniform_baseline")
    ub_attr = deep_shap_avg(X_test_c_tens, uniform_baseline, model)
    print("Start DeepSHAP for correct predictions with gaussian_baseline")
    gb_attr = deep_shap_avg(X_test_c_tens, gaussian_baseline, model)
    print("Start DeepSHAP for correct predictions with train_baseline")
    tb_attr = deep_shap_avg(X_test_c_tens, train_baseline, model)
    return bb_attr, mdb_attr, blb_attr, ub_attr, gb_attr, tb_attr

In [ ]:
def get_attr_scores(method, X_test_c_tens, black_baseline, blurred_baseline, uniform_baseline, gaussian_baseline, 
                train_baseline, model, X_min, X_max, columns):
    options = {
        'IG': calc_ig_attr,
        'DeepLIFT': calc_dl_attr,
        'DeepSHAP': calc_ds_attr
    }
    return options[method](X_test_c_tens, black_baseline, blurred_baseline, uniform_baseline, gaussian_baseline, 
                train_baseline, model, X_min, X_max, columns)

In [ ]:
def get_correct_predictions(model, X_test, X_test_tens, Y_test, averaging):
    np.random.seed(rs)
    random.seed(rs)
    
    Y_test_reset = Y_test.reset_index(drop=True)
    corr_idx = []

    predictions = model(X_test_tens.cuda())
    y_pred_label = torch.round(torch.sigmoid(predictions)) if averaging == 'binary' else torch.max(predictions.data, 1)[1]

    xs = np.asarray(X_test, dtype=np.float32)
    ys = np.asarray(Y_test, dtype=np.int) if averaging == 'binary' else np.asarray(Y_test, dtype=np.int) - 1

    for x, prediction, y in zip(enumerate(xs), y_pred_label, ys):
        if prediction == y:
            corr_idx.append(x[0])

    X_test_c = X_test[X_test.index.isin(corr_idx)]
    print(f"X_test of correct predictions shape: {X_test_c.shape}")
    Y_test_c = Y_test_reset[Y_test_reset.index.isin(corr_idx)] if averaging == 'binary' else Y_test_reset[Y_test_reset.index.isin(corr_idx)] - 1
    print(f"Y_test of correct predictions shape: {Y_test_c.shape}")
    unique, counts = np.unique(Y_test_c, return_counts=True)
    print(f"Label in Y_test of correct predictions ratio: \n {np.asarray((unique, counts)).T}")

    return X_test_c, Y_test_c

In [ ]:
def top_k_ablation_row_wise(model, X, y, baseline_attr, k, imputation, averaging):
    if k >= 1.0:
        amount = len(X.columns)
    else:
        amount = int(len(X.columns) * k)
    columns = X.columns
    attr_abs = np.abs(baseline_attr)
    if imputation.ndim == 1:
        for row_x, row_shap in zip(X.iterrows(), attr_abs):
            row_x = row_x[1]
            sorted_attr = np.argsort(-row_attr, axis=0)
            for i in range(0, amount):
                row_x[columns[sorted_attr[i]]] = imputation[columns[sorted_attr[i]]]
    elif imputation.ndim == 2:
        for row_x, row_attr, row_imp in zip(X.iterrows(), attr_abs, imputation.iterrows()):
            row_x, row_imp = row_x[1], row_imp[1]
            sorted_attr = np.argsort(-row_attr, axis=0)
            for i in range(0, amount):
                row_x[columns[sorted_attr[i]]] = row_imp[columns[sorted_attr[i]]]
                
    else:
        raise Exception('ndim of imputation too high, allowed 1 or 2')
    
    y_pred = model(torch.from_numpy(X.to_numpy(dtype=np.float32)).cuda())
    y_hat = torch.round(torch.sigmoid(y_pred)) if averaging == 'binary' else torch.max(y_pred.data, 1)[1]
    return f1_score(y, y_hat.cpu().data.numpy(), average=averaging)

In [ ]:
def random_k_ablation_row_wise(model, X, y, k, imputation, averaging):
    if k >= 1.0:
        amount = len(X.columns)
    else:
        amount = int(len(X.columns) * k)       
    columns = X.columns
    rnd_columns = random.sample(range(0, len(X.columns)), amount)
    if imputation.ndim == 1:
        for i in rnd_columns:
            X[columns[i]] = imputation[columns[i]]
    elif imputation.ndim == 2:
        for row_x, row_imp in zip(X.iterrows(), imputation.iterrows()):
            row_x, row_imp = row_x[1], row_imp[1]
            for i in rnd_columns:
                row_x[columns[i]] = row_imp[columns[i]]
    else:
        raise Exception('ndim of imputation too high, allowed 1 or 2')
    y_pred = model(torch.from_numpy(X.to_numpy(dtype=np.float32)).cuda())
    y_hat = torch.round(torch.sigmoid(y_pred)) if averaging == 'binary' else torch.max(y_pred.data, 1)[1]
    return f1_score(y, y_hat.cpu().data.numpy(), average=averaging)

In [ ]:
errors_dict = defaultdict(list)
datasets_names = ['har', 'compas', 'communities', 'fraud_detection', 'spambase']
datasets_paths = ['har', 'compas/', 'crime/', 'fraud_detection/', 'spambase/']
method = 'DeepLIFT'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

for dataset_name, dataset_path in zip(datasets_names, datasets_paths):
    np.random.seed(rs)
    torch.manual_seed(rs)
    random.seed(rs)
   
    fig = plt.figure(figsize=(12,8))
    plt.gca().xaxis.set_major_formatter(PercentFormatter(1))
    plt.grid()
        
    print(f"Start iteration for dataset '{dataset_name}'")
    data = f'datasets/{dataset_path}' if dataset_name == 'har' else pd.read_csv('datasets/' + dataset_path + dataset_name + '.' + 'csv')
    X_train, X_test, Y_train, Y_test = dataloader.prepare_dataset(dataset_name, data)
    X_train_tens = torch.tensor(X_train.to_numpy()).float()
    X_test_tens = torch.tensor(X_test.to_numpy()).float()
    
    unique, counts = np.unique(Y_train, return_counts=True)
    
    if (len(counts) > 2):
        averaging = 'macro'
        y_train_tens = torch.tensor(Y_train.to_numpy()).long() - 1
        output_dim = len(counts)
    else:
        averaging = 'binary'
        y_train_tens = torch.tensor(Y_train.to_numpy()).view(-1, 1).float()
        output_dim = 1
        
    dataset_tens = torch.utils.data.TensorDataset(X_train_tens, y_train_tens)
    train_iter = torch.utils.data.DataLoader(dataset_tens, batch_size, shuffle=False)
            
    print("Try to load the model..")
    model = nn_model.get_model(device, train_iter, X_train.shape[1], output_dim, averaging, learning_rate, num_epochs)
      
    print("Loading all baselines..")
    bun = BaselineUtilTensor()
    
    black_baseline = bun.create_black_baseline(X_train)
    blurred_baseline = bun.create_blurred_baseline(X_train, 10)
    uniform_baseline = bun.create_uniform_baseline(X_train)
    gaussian_baseline = bun.create_gaussian_baseline(X_train, 0.5)
    train_baseline = bun.create_train_baseline(X_train)
        
    print("Get correct predictions..")
    X_test_c, Y_test_c = get_correct_predictions(model, X_test, X_test_tens, Y_test, averaging)
    X_test_c_tens = torch.from_numpy(X_test_c.to_numpy(dtype=np.float32)).cuda()
    
    #ks for the top k ablation test
    ks = np.arange(start=0.0, stop=1.1, step=0.1)
    
    np.random.seed(rs)
    torch.manual_seed(rs)
    print("Get all attr. scores for the different baselines..")
    bb_attr, mdb_attr, blb_attr, ub_attr, gb_attr, tb_attr = get_attr_scores(method, X_test_c_tens, black_baseline,
                                                                             blurred_baseline, uniform_baseline,
                                                                             gaussian_baseline, train_baseline, model,
                                                                            X_test_c.min(), X_test_c.max(), X_test_c.columns)
    attribute_scores = {
          "Constant Baseline": bb_attr,
          "Maximum Distance Baseline": mdb_attr,
          "Blurred Baseline": blb_attr,
          "Uniform Baseline": ub_attr,
          "Gaussian Baseline": gb_attr,
          "Expectation Baseline": tb_attr
    }
        
    print("Start Top-K Ablation..")
    imputation = imp.load_imputation(imputation_typ, X_test_c)
    test_errors = []
    for name, score in attribute_scores.items():
        for k in ks:
            key = name + str(round(k,1))
            if method == 'DeepSHAP':
                error = top_k_ablation_row_wise(model, X_test_c.copy(deep=True), Y_test_c, score, k, imputation, averaging)
                test_errors.append(error)
                #For the avg plots
                errors_dict[key].append(error)
            else:
                error = top_k_ablation_row_wise(model, X_test_c.copy(deep=True), Y_test_c, score.cpu().data.numpy(), k, imputation, averaging)
                test_errors.append(error)
                errors_dict[key].append(error)
                
        plt.plot(ks, test_errors, label=name, linewidth=2.5, alpha=0.7)
        test_errors.clear()
    
    for k in ks:
        key = 'Random Ablation' + str(round(k,1))
        error = random_k_ablation_row_wise(X_test_c.copy(deep=True), Y_test_c, k, imputation, averaging)
        test_errors.append(error)
        errors_dict[key].append(error)
                
    plt.plot(ks, test_errors, label='Random Ablation', linewidth=2.5, alpha=0.7)
    test_errors.clear()
        
    plt.xlabel('Ablation percentage of the most important features determined by the baseline',fontsize=14)
    plt.ylabel('F1 score as fraction of original F1 score', fontsize=14)
    plt.legend(prop={'size': 13})
    fig.savefig(f"plots/top_k_test_{method}_{dataset_name}_nn.pdf", dpi=fig.dpi,
                format='pdf', bbox_inches='tight')
    plt.show()
    

# Create overall AVG plots

In [ ]:
baseline_names = ["Constant Baseline",
"Maximum Distance Baseline",
"Blurred Baseline",
"Uniform Baseline",
"Gaussian Baseline",
"Expectation Baseline",
"Random Ablation"]

test_errors = defaultdict(list)
test_std = defaultdict(list)

for b in baseline_names:
    for k in ks:
        key = b + str(round(k,1))
        test_errors[b].append(np.mean(errors_dict[key]))
        test_std[b].append(np.std(errors_dict[key]))

In [ ]:
fig = plt.figure(figsize=(12,8))
plt.gca().xaxis.set_major_formatter(PercentFormatter(1))
plt.grid()

for b in baseline_names:
    y = np.asarray(test_errors[b])
    std = np.asarray(test_std[b])
    plt.plot(ks, y, label=b, linewidth=2.5, alpha=0.7)
    plt.fill_between(ks, y-(std/2), y+(std/2), alpha=0.2)

plt.xlabel('Ablation percentage of the most important features determined by the baseline',fontsize=14)
plt.ylabel('F1 score as fraction of original F1 score', fontsize=14)
plt.legend(prop={'size': 13})
fig.savefig(f"plots/top_k_test_AVG_{method}_w_shader.pdf", dpi=fig.dpi,
            format='pdf', bbox_inches='tight')

In [ ]:
fig = plt.figure(figsize=(12,8))
plt.gca().xaxis.set_major_formatter(PercentFormatter(1))
plt.grid()

for b in baseline_names:
    y = np.asarray(test_errors[b])
    std = np.asarray(test_std[b])
    plt.plot(ks, y, label=b, linewidth=2.5, alpha=0.7)

plt.xlabel('Ablation percentage of the most important features determined by the baseline',fontsize=14)
plt.ylabel('F1 score as fraction of original F1 score', fontsize=14)
plt.legend(prop={'size': 13})
fig.savefig(f"plots/top_k_test_AVG_{method}.pdf", dpi=fig.dpi,
            format='pdf', bbox_inches='tight')